In [ ]:
## Import libraries
import json
import glob
import os
import re 
import requests
import numpy as np
import processing
import sys
import qgis
from qgis.core import *
from processing.core.Processing import Processing

In [ ]:
localPath = ""## put your local path 

In [ ]:
QgsApplication.setPrefixPath('/Applications/QGIS3.16.app', True)
qgs = QgsApplication([], False)
qgs.initQgis()
Processing.initialize()

In [ ]:
outfile = open(localPATH +'.js', 'a')
outfile.write("var planetImages = [")
folderPath = "/Volumes/Projects/NASA/Imagery/Galapagos/PlanetScope"
losFolders = os.listdir(folderPath)



In [ ]:
## find the Tiff and the JSON in PlanetScope
for elFolder in losFolders:
    path = os.path.join(folderPath,elFolder)
    size = 0
    max_size = 0
    max_file ="" 
    # walking through the entire folder, 
    # including subdirectories 

    for folder, subfolders, files in os.walk(path): 

        # checking the size of each file 
        for file in files: 
            size = os.stat(os.path.join( folder, file  )).st_size 
            match = re.search("\.json$", file) 

        # if match is found 
            if match: 
                jsFile = os.path.join( folder, file  )
    #             print("The file ending with .json is:", file) 


            # updating maximum size 
            if size>max_size: 
                max_size = size 
                max_file = os.path.join( folder, file ) 


    ## Opening the JS File and getting the interesting data            
    f = open(jsFile, "r") 
    data = json.loads(f.read())

    xy = data['geometry']['coordinates']
    ID = data['id']
    date = data['properties']['acquired']
    provider = data['properties']['provider']
    cloudCover = data['properties']['cloud_cover']

    outJSON = ({
        'ID': ID,
        'date': date,
        'provider': provider,
        'cloudCover':cloudCover,
        'coordinate':xy 
    })

    outfile.write(str(outJSON))
    outfile.write(",")
    outfile.write("\n")

    newPath = os.path.join(localPath, ID) 
    os.mkdir(newPath)
    rlayer = QgsRasterLayer(max_file, "myRaster")
    project = QgsProject.instance().addMapLayer(rlayer, True)

    transform = QgsCoordinateTransform(rlayer.crs(),
                                       QgsCoordinateReferenceSystem("EPSG:4326"), QgsProject.instance())
    # transform.transform(rlayer.extent())
    processing.run("qgis:tilesxyzdirectory", {
        'DPI' : 96, 
        'EXTENT' : transform.transform(rlayer.extent()), 
        'METATILESIZE' : 4, 
        'OUTPUT_DIRECTORY' : newPath, 
        'OUTPUT_HTML' : localPath+ID+".html", 
        'QUALITY' : 75, 
        'TILE_FORMAT' : 0, 
        'TILE_HEIGHT' : 256, 
        'TILE_WIDTH' : 256, 
        'TMS_CONVENTION' : False, 
        'ZOOM_MAX' : 12, 
        'ZOOM_MIN' : 10 
    }) # 'TMS_CONVENTION' : False, 


In [ ]:
outfile.write("]")
outfile.close() 